In [32]:
import pandas as pd
import os
import requests
import io
from urllib.request import urlopen
import zipfile
import warnings
import unidecode
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
from urllib.request import urlopen
import zipfile
import difflib
import basedosdados as bd

warnings.filterwarnings("ignore")

In [3]:
for url in range(2013, 2025):
    url = f"https://portaldatransparencia.gov.br/download-de-dados/receitas/{url}"
    if requests.get(url).status_code == 200:

        r = urlopen(url)
        z = zipfile.ZipFile(io.BytesIO(r.read()))
        z.extractall("/home/tricktx/dados/br_cgu_receitas")

In [33]:
def to_partitions(
    data: pd.DataFrame,
    partition_columns: List[str],
    savepath: str,
    file_type: str = "csv",
):
    """Save data in to hive patitions schema, given a dataframe and a list of partition columns.
    Args:
        data (pandas.core.frame.DataFrame): Dataframe to be partitioned.
        partition_columns (list): List of columns to be used as partitions.
        savepath (str, pathlib.PosixPath): folder path to save the partitions.
        file_type (str): default to csv. Accepts parquet.
    Exemple:
        data = {
            "ano": [2020, 2021, 2020, 2021, 2020, 2021, 2021,2025],
            "mes": [1, 2, 3, 4, 5, 6, 6,9],
            "sigla_uf": ["SP", "SP", "RJ", "RJ", "PR", "PR", "PR","PR"],
            "dado": ["a", "b", "c", "d", "e", "f", "g",'h'],
        }
        to_partitions(
            data=pd.DataFrame(data),
            partition_columns=['ano','mes','sigla_uf'],
            savepath='partitions/',
        )
    """

    if isinstance(data, (pd.core.frame.DataFrame)):
        savepath = Path(savepath)
        # create unique combinations between partition columns
        unique_combinations = (
            data[partition_columns]
            # .astype(str)
            .drop_duplicates(subset=partition_columns).to_dict(orient="records")
        )

        for filter_combination in unique_combinations:
            patitions_values = [
                f"{partition}={value}"
                for partition, value in filter_combination.items()
            ]

            # get filtered data
            df_filter = data.loc[
                data[filter_combination.keys()]
                .isin(filter_combination.values())
                .all(axis=1),
                :,
            ]
            df_filter = df_filter.drop(columns=partition_columns)

            # create folder tree
            filter_save_path = Path(savepath / "/".join(patitions_values))
            filter_save_path.mkdir(parents=True, exist_ok=True)

            if file_type == "csv":
                # append data to csv
                file_filter_save_path = Path(filter_save_path) / "data.csv"
                df_filter.to_csv(
                    file_filter_save_path,
                    sep=",",
                    encoding="utf-8",
                    na_rep="",
                    index=False,
                    mode="a",
                    header=not file_filter_save_path.exists(),
                )
            elif file_type == "parquet":
                # append data to parquet
                file_filter_save_path = Path(filter_save_path) / "data.parquet"
                df_filter.to_parquet(
                    file_filter_save_path, index=False, compression="gzip"
                )
    else:
        raise BaseException("Data need to be a pandas DataFrame")

In [35]:
for path in range(2013, 2025):
    path = f"/home/tricktx/dados/br_cgu_receitas/{path}_Receitas.csv"
    df = pd.read_csv(path, sep=";", encoding="latin1")

    df.columns = [unidecode.unidecode(col) for col in df.columns]
    df.columns = [col.replace(" ", "_").lower() for col in df.columns]

    valores = [
        "valor_previsto_atualizado",
        "valor_lancado",
        "valor_realizado",
        "percentual_realizado",
    ]

    for x in valores:
        df[x] = df[x].str.replace(",", ".").astype(float)

    to_partitions(
        data=df,
        partition_columns=["ano_exercicio"],
        savepath="/home/tricktx/dados/br_cgu_receitas/partitions/",
    )

In [28]:
df.columns = [unidecode.unidecode(col) for col in df.columns]
df.columns = [col.replace(" ", "_").lower() for col in df.columns]

In [30]:
valores = [
    "valor_previsto_atualizado",
    "valor_lancado",
    "valor_realizado",
    "percentual_realizado",
]

for x in valores:
    df[x] = df[x].str.replace(",", ".").astype(float)

In [31]:
df.head()

,codigo_orgao_superior,nome_orgao_superior,codigo_orgao,nome_orgao,codigo_unidade_gestora,nome_unidade_gestora,categoria_economica,origem_receita,especie_receita,detalhamento,valor_previsto_atualizado,valor_lancado,valor_realizado,percentual_realizado,data_lancamento,ano_exercicio
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr","BENS,DIR.VAL.PERD.POD.PUB.-CRIME COMUM-PRINC.",0.0,0.0,8.24,0.0,24/02/2023,2023
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Serviços Administrativos e Comerciais Gerais,INSCR.EM CONCURSOS E PROC.SELETIVOS-PRINCIPAL,0.0,0.0,113940.00,0.0,26/01/2023,2023
2,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr","BENS,DIR.VAL.PERD.POD.PUB.-CRIME COMUM-PRINC.",0.0,0.0,4650.77,0.0,31/05/2023,2023
3,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Serviços Administrativos e Comerciais Gerais,INSCR.EM CONCURSOS E PROC.SELETIVOS-PRINCIPAL,0.0,0.0,112140.00,0.0,23/02/2023,2023
4,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receitas Correntes - a classificar,Receitas Correntes - a classificar,Receitas Correntes - a classificar,0.0,0.0,0.00,0.0,24/01/2023,2023
